In [ ]:
import gc
import torch
import time

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.chains import RetrievalQA
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker

In [2]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
model = ChatOpenAI(model='gpt-5-mini', temperature=0)
hf_model = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-base')

In [3]:
loader = PyPDFLoader('./data/투자설명서.pdf')
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
vectorstore = Chroma(collection_name='split_parents', embedding_function=embeddings)
docstore = InMemoryStore()

In [5]:
retriever = ParentDocumentRetriever(
    docstore=docstore,
    vectorstore=vectorstore,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter
)

docs = retriever.add_documents([Document(page_content=full_text)])

In [6]:
reranker = FlashrankRerank(model='ms-marco-MiniLM-L-12-v2', top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever
)

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=compression_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [8]:
query = '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야? 참고로 합병된 회사의 정보를 찾으면 안돼.'
result = qa_chain.invoke(query)

print(f"[답변]: {result['result']}")
print("\n--- [참조된 문서]: ---")
for doc in result['source_documents']:
    print(doc.page_content)
    print("-" * 20)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[답변]: 2022년(제17기) 별도재무제표 기준 영업손실은 14,911,333,804원입니다.

--- [참조된 문서]: ---
1) 최근 3년간 요약 재무상태표
(단위 : 원)
구분 2023년 2022년 2021년
 유동자산 976,148,260 1,751,083,731 1,809,632,798
 비유동자산 - - -
자산총계 976,148,260 1,751,083,731 1,809,632,798
 유동부채 1,162,562,658 1,110,583,715 1,400,247,015
 비유동부채 295,000,000 300,000,000 -
부채총계 1,457,562,658 1,410,583,715 1,400,247,015
 자본금 500,000,000 500,000,000 500,000,000
 자본잉여금 - - -
 자본조정 - - -
 이익잉여금 -981,414,398 -159,499,984 -90,614,217
자본 총계 -481,414,398 340,500,016 409,385,783
부채 및 자본총계 976,148,260 1,751,083,731 1,809,632,798
2) 최근 3년간 요약 손익계산서
(단위 : 원)
구분 2023년 2022년 2021년
 매출액 5,576,793,616 3,607,082,538 3,394,446,212
 매출원가 5,648,941,378 3,037,093,479 3,325,923,511
 매출총이익 -72,147,762 569,989,059 68,522,701
 판매비와관리비 552,549,508 620,173,548 90,358,437
 영업이익(손실) -624,697,270 -50,184,489 -21,835,736
 영업외 수익 161,753,839 857,457 16,070,484
 영업외 비용 358,970,983 19,558,735 29,344,707
 법인세비용차감전 순이익 -821,914,414 -68,885,767 -35,109,959
 법인세비용 - - -
 당기순이익 -821,914,414 

In [9]:
reranker = CrossEncoderReranker(model=hf_model, top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever
)

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=compression_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [11]:
query = '이 회사의 2022년 영업이익 혹은 영업손실이 정확히 얼마야? 참고로 합병된 회사의 정보를 찾으면 안돼.'
result = qa_chain.invoke(query)

print(f"[답변]: {result['result']}")
print("\n--- [참조된 문서]: ---")
for doc in result['source_documents']:
    print(doc.page_content)
    print("-" * 20)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[답변]: 2022년 영업손실은 14,911,333,804원입니다. (영업이익(손실) 항목에 “(14,911,333,804)”으로 표기된 값이며, 합병된 회사의 재무정보(-50,184,489원 등)는 제외한 수치입니다.)

--- [참조된 문서]: ---
당사의 주요 사업은 온라인 채널(이커머스)을 활용한 B2C 유통업으로, 주
력 상품 카테고리는 식품입니다. 이에 식품위생법, 축산물위생관리법, 전자
상거래 등에서의 소비자보호에 관한 법규 등을 적용 받으며, 이를 준수할
의무가 있습니다.
당사는 관련 법규를 준수하며 영업활동을 진행하고 있으나, 농축수산물을
비롯하여 당사가 주로 취급하는 식품 카테고리 특성상 예기치 못한 위생 관
련 이슈가 발생할 수 있습니다. 유통과정에서의 상품 변질 등 당사의 관리
책임 미비로 관련 법률에 저촉될 경우 벌금, 과태료, 영업 정지, 당사
브랜드 가치 훼손, 고객 보상 비용, 법률 비용의 발생 등을 초래
할 수 있으며, 이는 당사의 재무실적에 부정적인 영향을 미칠 수
있으니 투자자 여러분들께서는 이 점 유의하시기 바랍니다.
[종합]
결론적으로 당사는 기존에 본사에서 영위하고 있던 사업과는 연관
성이 낮은 사업에 신규 진출한 상황이므로, 해당 사업에서 발생
가능한 위험 요소들을 통제하는 데 있어서 기존 경영진들의 관
련 산업군에서의 경험이 부족한 상황입니다. 본 증권신고서 "제1부
.III.투자위험요소-2.회사위험-가." 부분에 기재된 바와 같이, 이를 극복하
고자 일부 인력을 승계하고, 사업 전개가 원활하도록 노력하고 있으나, 사
업의 성패를 장담할 수 없고, 신규사업 진출이 당사에 악영향을
미치는 결과를 초래할 수 있습니다. 투자자 여러분들께서는 이 점을
반드시 유의하여 주시기 바랍니다.
회사위험
[가. 매출부진 및 지속적 손실 발생 위험]
당사는 매출이 부진한 가운데, 지속적으로 신약후보물질의 발굴과 보유 파
이프라인의 적응증 증가를 위하여 다양한 비임상 및 임상시험을 준비하고
있으며 이에 따라 관련비용의 지출이 꾸준히 발생